In [ ]:
import torch
import numpy as np
from simple_RNN import *

In [ ]:
filename = 'input.txt'
#filename = 'wodehouse_right_ho_jeeves.txt'
data_raw = open(filename, 'r').read() # should be simple plain text file
data_raw = data_raw.lower()

data, vocab_size, idx_to_char = data_from_text(data_raw)
data_one_hot = np.eye(vocab_size)[data]


In [ ]:
seq_length = 25 # this is how much of the data we sample before updating the params
hidden_size = 50 # size of the hidden state vector
learning_rate = 0.1
num_epochs = 50

verbose = True

In [ ]:
class MyModel(torch.nn.Module):
    def __init__(self, vocab_size, hidden_size, initial_params):
        super(MyModel, self).__init__()

        # create recurrent and linear layers
        rnn = torch.nn.RNN(input_size=vocab_size,hidden_size=hidden_size)
        linear = torch.nn.Linear(hidden_size, vocab_size)

#        # set the weight tensors
#         dtype = torch.float
#         tensor_parameters = {name:torch.tensor(param, dtype=dtype) for name,param in parameters.items()}
#         rnn.bias_ih_l0.data.fill_(0)
#         rnn.weight_hh_l0.data = tensor_parameters['W']
#         rnn.weight_ih_l0.data = tensor_parameters['U']
#         rnn.bias_hh_l0.data = tensor_parameters['bh']
#         linear.weight.data = tensor_parameters['V']
#         linear.bias.data = tensor_parameters['by']
        
        self.rnn = rnn
        self.linear = linear
        self.h_prev = torch.zeros([1, 1, hidden_size])
        
    def forward(self, input):
        h, h_prev = self.rnn(inputs, self.h_prev)
        out = self.linear(h)
        self.h_prev = h_prev.detach()
        return out

In [ ]:
parameters = initialize_parameters(hidden_size, vocab_size)
model = MyModel(vocab_size, hidden_size, parameters)
model

In [ ]:
# initialize parameters
#parameters = initialize_parameters(hidden_size, vocab_size)
dtype = torch.float
data_length = len(data)
seqs_per_epoch = int(data_length/seq_length)  # will this work if data/seq divides exactly?

loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')

#optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-1)

data3D = np.expand_dims(data_one_hot, 1)
for epoch in range(num_epochs):
    # initialize hidden  state    
    h_prev = torch.zeros([1, 1, hidden_size])
    model.h_prev = h_prev
    if verbose and epoch != 0:
        print('epoch: {}, loss: {}'.format(epoch, loss))

    for i in range(seqs_per_epoch):
        start = i*seq_length
        end = i*seq_length+seq_length
        end = min(end, data_length-1)

        inputs_raw = data3D[start:end]
        targets_raw = data[start+1:end+1]
             
        # both inputs and targets are (seq_length, 1, vocab_size)
        inputs = torch.tensor(inputs_raw, dtype=dtype)
        targets = torch.tensor(targets_raw, dtype=torch.long)
                
        out = model(inputs)
        
        out2 = out.view((seq_length, vocab_size))
        loss = loss_fn(out2, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print( i, loss.item())
